# Problem Statement
__You have been tasked with developing an AI system that can analyze a PDF file and generate a summary. of its contents. Additionally, the system should be integrated with a Chat GPT API to allow users to ask. questions related to the PDF file and received relevant answers__.

# Objective:
__The objective of this assignment is to develop an AI system that can__:
1. Analyze a PDF file and generate a summary of its contents.
2. Integrate with a Chat GPT API to answer user questions related to the PDF file.
3. Suggest questions based on the PDF file content.
4. Provide successive questions based on user questions.

# Requirements:
__The following are the requirements for the AI system__:
1. The system should be able to analyze a PDF file and generate a summary of its contents using NLP techniques.
2. The system should be integrated with a Chat GPT API to allow users to ask questions related to the PDF file and receive relevant answers.
3. The system should be able to suggest questions based on the content of the PDF file.
4. The system should provide successive questions based on user questions to allow for a more natural conversation flow.
5. The system should be able to handle multiple users concurrently.
6. The system should be able to handle errors and exceptions gracefully.

# Steps to Run-:
1. __Install the modules__. 
2. __Replace the openai key on line 3__. 
3. __Replace the Pdf file path on line 80__. 

# Module Installation

In [ ]:
!pip install openai
!pip install pdfplumber

# Importing Modules

In [ ]:
import openai
import pdfplumber

openai.api_key = 'Open_AI_Key'

# New Approach

In [ ]:
def extract_pdf_text(file_path):
    with pdfplumber.open(file_path) as pdf:
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
        return text

def generate_summary(text):
    prompt = f"Summarize:\n\n{text[:1000]}"
    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        max_tokens=300,
        temperature=0.5,
        n=1,
        stop=None
    )
    summary = response.choices[0].text.strip()
    return summary

def generate_questions(prompt, num_questions):
    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt + "\nQ:",
        max_tokens=200,
        temperature=0.5,
        n=num_questions,
        stop=None
    )
    questions = [choice.text.strip() for choice in response.choices]
    return questions

def generate_answer(question, context):
    prompt = f"{context}\n{question}"
    response = openai.Completion.create(
        engine='text-davinci-003',
        prompt=prompt,
        max_tokens=200,
        temperature=0.5,
        n=1,
        stop=None
    )
    answer = response.choices[0].text.strip()
    return answer

def process_pdf(file_path):
    try:
        pdf_text = extract_pdf_text(file_path)
        summary = generate_summary(pdf_text)
        num_initial_questions = 5
        initial_questions = generate_questions(summary, num_initial_questions)
        
        print(f"\nSummary for file '{file_path}':\n{summary}")
        print("\nTop 5 Questions:")
        for i, question in enumerate(initial_questions):
            print(f"{i+1}. {question}")

        # User selects a question
        selected_question_index = int(input("\nSelect a question (1-5): "))
        selected_question = initial_questions[selected_question_index - 1]
        
        # Generate answer based on the selected question
        answer = generate_answer(selected_question, summary)
        
        print("\nUser Selection:")
        print(f"Question: {selected_question}")
        print(f"Answer: {answer}")

        # Ask user if they want to generate further questions based on the answer
        further_questions_input = input("\nDo you want to generate further questions based on the answer? (yes or no): ")
        if further_questions_input.lower() == 'yes':
            followup_questions = generate_questions(answer, num_initial_questions)
            print("\nFollow-up Questions based on the answer:")
            for i, question in enumerate(followup_questions):
                print(f"{i+1}. {question}")

            output = {
                'file_path': file_path,
                'summary': summary,
                'selected_question': selected_question,
                'answer': answer,
                'followup_questions': followup_questions
            }
        else:
            output = {
                'file_path': file_path,
                'summary': summary,
                'selected_question': selected_question,
                'answer': answer
            }

        return output

    except Exception as e:
        error_message = f"An error occurred: {str(e)}"
        return {'file_path': file_path, 'error': error_message}

# Specify the paths to the PDF files
pdf_paths = ['Analysis Report_ Comparison of Different Models for Dispense Amount Prediction.pdf']

# Process the PDF files
results = []
for pdf_path in pdf_paths:
    result = process_pdf(pdf_path)
    results.append(result)